In [2]:
import tifffile as tiff
import numpy as np
from PIL import Image

In [34]:
img = tiff.imread('anomaly_maps/EfficientNetV2SAutoencoder_224/bottle/test/broken_large/010.tiff')

In [35]:
img

array([[2.03946198e-04, 2.00474780e-04, 1.93531931e-04, ...,
        1.07035594e-04, 1.09928449e-04, 1.11374873e-04],
       [2.00185488e-04, 1.96714071e-04, 1.90060498e-04, ...,
        1.05589170e-04, 1.08482018e-04, 1.09928449e-04],
       [1.92953361e-04, 1.89481929e-04, 1.83117649e-04, ...,
        1.02696315e-04, 1.05299885e-04, 1.07035594e-04],
       ...,
       [6.07499305e-06, 5.78570780e-06, 5.49642255e-06, ...,
        9.45963257e-05, 9.83570353e-05, 1.00382029e-04],
       [6.07499305e-06, 6.07499305e-06, 5.78570780e-06, ...,
        9.83570353e-05, 1.02407030e-04, 1.04142739e-04],
       [6.36427876e-06, 6.07499305e-06, 5.78570780e-06, ...,
        1.00382029e-04, 1.04142739e-04, 1.06167739e-04]], dtype=float32)

In [25]:
nom_img = (img - np.min(img))/(np.max(img) - np.min(img))

In [38]:
len(np.where(img > 0.7)[0])

446

In [5]:
img = Image.open('mvtec_anomaly_detection_224/bottle/ground_truth/broken_large/000_mask.png')

In [7]:
np_img = np.asarray(img)

In [16]:
(np_img == 1).sum()

0

In [ ]:
from torchvision.models import swin_b, Swin_B_Weights
from torchsummary import summary

model = swin_b(Swin_B_Weights.DEFAULT)


/home/leejinhyeok/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/swin_b-68c6b09e.pth" to /home/leejinhyeok/.cache/torch/hub/checkpoints/swin_b-68c6b09e.pth
100%|██████████| 335M/335M [00:30<00:00, 11.4MB/s] 


In [4]:
summary(model.features, (3,224,224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 56, 56]           6,272
           Permute-2          [-1, 56, 56, 128]               0
         LayerNorm-3          [-1, 56, 56, 128]             256
         LayerNorm-4          [-1, 56, 56, 128]             256
ShiftedWindowAttention-5          [-1, 56, 56, 128]               0
   StochasticDepth-6          [-1, 56, 56, 128]               0
         LayerNorm-7          [-1, 56, 56, 128]             256
            Linear-8          [-1, 56, 56, 512]          66,048
              GELU-9          [-1, 56, 56, 512]               0
          Dropout-10          [-1, 56, 56, 512]               0
           Linear-11          [-1, 56, 56, 128]          65,664
          Dropout-12          [-1, 56, 56, 128]               0
  StochasticDepth-13          [-1, 56, 56, 128]               0
SwinTransformerBlock-14          [-

In [ ]:
import torch
import torch.nn as nn
class Decoder(nn.Module):
    def __init__(self, channels, kernel_size=4, stride=2, padding=1):
        super(Decoder, self).__init__()
        
        def conv_block(in_channels, out_channels):
            return nn.Sequential(
                nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
                nn.BatchNorm2d(out_channels),
                nn.ReLU()
            )
            
        layers = [conv_block(channels[i], channels[i + 1]) for i in range(len(channels) - 1)]
        layers.append(nn.Sigmoid())
        self.decoder = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.decoder(x)
class SwinT(nn.Module):
    def __init__(self):
        super(SwinT, self).__init__()
        self.encoder = swin_b(Swin_B_Weights.DEFAULT).features
        
        self.decoder = Decoder([1024, 512, 256, 128, 64, 3])
        
    def forward(self, x):
        x = self.encoder(x)
        x = torch.permute(x,(0,2,3,1))
        x = self.decoder(x)
        return x

In [12]:
model = SwinT()

/home/leejinhyeok/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [13]:
model

SwinT(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (1): Permute()
      (2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (1): Sequential(
      (0): SwinTransformerBlock(
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): ShiftedWindowAttention(
          (qkv): Linear(in_features=128, out_features=384, bias=True)
          (proj): Linear(in_features=128, out_features=128, bias=True)
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=512, out_features=128, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (1): SwinTransformerBlock(
        (

In [14]:
summary(model, (3,224,224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 56, 56]           6,272
           Permute-2          [-1, 56, 56, 128]               0
         LayerNorm-3          [-1, 56, 56, 128]             256
         LayerNorm-4          [-1, 56, 56, 128]             256
ShiftedWindowAttention-5          [-1, 56, 56, 128]               0
   StochasticDepth-6          [-1, 56, 56, 128]               0
         LayerNorm-7          [-1, 56, 56, 128]             256
            Linear-8          [-1, 56, 56, 512]          66,048
              GELU-9          [-1, 56, 56, 512]               0
          Dropout-10          [-1, 56, 56, 512]               0
           Linear-11          [-1, 56, 56, 128]          65,664
          Dropout-12          [-1, 56, 56, 128]               0
  StochasticDepth-13          [-1, 56, 56, 128]               0
SwinTransformerBlock-14          [-

ModuleNotFoundError: No module named 'monai'

In [6]:
import monai

ModuleNotFoundError: No module named 'monai'

In [1]:
import torch
import monai

print("PyTorch version:", torch.__version__)
print("MONAI version:", monai.__version__)


PyTorch version: 2.5.0+cu124
MONAI version: 1.4.0


In [6]:
from monai.networks.nets import SwinUNETR

model = SwinUNETR(
    img_size=(224, 224, 224),     
    in_channels=3,             # 입력 채널 수 (예: 흑백 이미지면 1, 컬러면 3)
    out_channels=3,            # 출력 채널 수 (예: 이진 분할일 경우 2)
    feature_size=768,           # 모델의 기본 feature 크기
    use_checkpoint=True )

: 